In [ ]:
#default_exp plot

# 02_plot

> Contains plotting helper functions

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
from fastai2.tabular.all import *
from fastai2.vision.all import *

In [ ]:
#export
@typedispatch
def plot(x:TfmdDL, interp, combs, combs_l, tab, i=None, boxes=None, cols=None, rows=None, ranges=None, figsize=(12,12), cut_off=100):
  "Plot tabular graphs"
  y = 0
  if ranges[i] < boxes:
    cols = math.ceil(math.sqrt(ranges[i]))
    rows = math.ceil(ranges[i]/cols)
  if ranges[i]<4 or boxes < 4:
    cols, rows = 2, 2
  else:
    cols = math.ceil(math.sqrt(boxes))
    rows = math.ceil(boxes/cols)
  fig, ax = plt.subplots(rows, cols, figsize=figsize)

  [axi.set_axis_off() for axi in ax.ragel()]
  for j, idx in enumerate(combs_l):
    if boxes < y+1 or y > ranges[i]:
      break
    row = (int)(y/cols)
    col = x % cols
    img, lbl = x.dataset[idx]
    fn = x.items[idx]
    fn = re.search('([^/*]+)_\d+.*$', str(fn)).group(0)
    img.show(ctx=ax[row,col])
    x+=1
  plt.show(fig)
  plt.tight_layout()

In [ ]:
#export
@typedispatch
def plot(x:TabDataLoader, interp, idxs, combs, tab, i=None, boxes=None, cols=None, rows=None, ranges=None, figsize=(12,12), cut_off=100):
  "Plot tabular graphs"
  if boxes is not None:
    fig, ax = plt.subplots(boxes, figsize=figsize)
  else:
    fig, ax = plot.subplots(cols, rows, figsize=figsize)
  fig.subplots_adjust(hspace=.5)
  titles = ['Original'] + combs
  for j, y in enumerate(idxs):
    title = f'{titles[j]} {tab} distribution'
    if boxes is None:
      row = int(j/cols)
      col = j%row
    if tab in x.cat_names:
      vals = pd.value_counts(y[tab].values)
      if boxes is not None:
        if vals.nunique() < 10:
            fig = vals.plot(kind='bar', title=title, ax=ax[j], rot=0, width=.75)
        elif vals.nunique() > cut_off:
          print(f'Number of unique values ({vals.nunique()}) is above {cut_off}')
        else:
          fig = vals.plot(kind='barh', title=title, ax=ax[j], width=.75)
      else:
        fig = vals.plot(kind='barh', title=title, ax=ax[row,col], width=.75)
    else:
      vals = y[tab]
      if boxes is not None:
        fig = vals.plot(kind='hist', ax=ax[j], title=title, y='Frequency')
      else:
        fig = vals.plot(kind='hist', ax=ax[row+1, col], title=title, y='Frequency')
      fig.set_ylabel('Frequency')
      if len(set(vals)) > 1:
        vals.plot(kind='kde', ax=fig, title=title, secondary_y=True)
      else:
        print("Less than two unique values, cannot graph the KDE")